In [1]:
import numpy as np
import pandas as pd

# stuck with the CPU version for now...
high_exp = pd.read_csv("high_exp.csv", index_col=0)
low_exp = pd.read_csv("low_exp.csv", index_col=0)

data_df = pd.concat([high_exp, low_exp], axis=0)

def string_to_matrix(color_string):
    color_string = str(color_string)
    list_of_strings = color_string.replace('[', '').replace(']', '').split('\n')
    list_of_lists = [channels.strip().split() 
                     for channels in list_of_strings
                     if 'nan' not in list_of_strings
                    ]
    remaining_pad = 181 - len(list_of_lists)
    while remaining_pad > 0:
        list_of_lists.append(list([0, 0, 0, 0]))
        remaining_pad = remaining_pad - 1
    
    return np.array(list_of_lists).astype(np.float)

data_df['color_matrix'] = data_df['color_matrix'].apply(string_to_matrix)

# create train test split
from sklearn.model_selection import train_test_split

X = np.zeros((22604, 181, 4))
for idx, colors in enumerate(data_df['color_matrix'].values):
    X[idx, :, :] = colors
    
y = data_df['class'].values

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [15]:
# simple model per Yoon Kim (2014)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, GlobalMaxPooling1D

model = Sequential()
model.add(Conv1D(10, 3, activation='selu', input_shape=(181, 4)))
model.add(GlobalMaxPooling1D())
# model.add(Flatten())
# model.add(Dense(32))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=50, epochs=100,
          validation_data=(x_test, y_test), verbose=2)

Train on 15822 samples, validate on 6782 samples
Epoch 1/100
 - 3s - loss: 0.7092 - acc: 0.5067 - val_loss: 0.6918 - val_acc: 0.5215
Epoch 2/100
 - 2s - loss: 0.6933 - acc: 0.5221 - val_loss: 0.6906 - val_acc: 0.5270
Epoch 3/100
 - 2s - loss: 0.6912 - acc: 0.5267 - val_loss: 0.6897 - val_acc: 0.5385
Epoch 4/100
 - 2s - loss: 0.6911 - acc: 0.5263 - val_loss: 0.6903 - val_acc: 0.5351
Epoch 5/100
 - 2s - loss: 0.6907 - acc: 0.5274 - val_loss: 0.6888 - val_acc: 0.5469
Epoch 6/100
 - 2s - loss: 0.6902 - acc: 0.5269 - val_loss: 0.6899 - val_acc: 0.5348
Epoch 7/100
 - 2s - loss: 0.6897 - acc: 0.5300 - val_loss: 0.6883 - val_acc: 0.5492
Epoch 8/100
 - 2s - loss: 0.6895 - acc: 0.5357 - val_loss: 0.6876 - val_acc: 0.5472
Epoch 9/100
 - 2s - loss: 0.6890 - acc: 0.5335 - val_loss: 0.6872 - val_acc: 0.5531
Epoch 10/100
 - 2s - loss: 0.6888 - acc: 0.5336 - val_loss: 0.6871 - val_acc: 0.5515
Epoch 11/100
 - 2s - loss: 0.6889 - acc: 0.5401 - val_loss: 0.6869 - val_acc: 0.5537
Epoch 12/100
 - 2s - loss

Epoch 97/100
 - 2s - loss: 0.6819 - acc: 0.5623 - val_loss: 0.6819 - val_acc: 0.5689
Epoch 98/100
 - 2s - loss: 0.6817 - acc: 0.5645 - val_loss: 0.6815 - val_acc: 0.5674
Epoch 99/100
 - 2s - loss: 0.6811 - acc: 0.5579 - val_loss: 0.6820 - val_acc: 0.5680
Epoch 100/100
 - 2s - loss: 0.6823 - acc: 0.5590 - val_loss: 0.6814 - val_acc: 0.5656


In [16]:
model = Sequential()
model.add(Conv1D(100, 3, activation='selu', input_shape=(181, 4)))
model.add(GlobalMaxPooling1D())
# model.add(Flatten())
# model.add(Dense(32))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=50, epochs=100,
          validation_data=(x_test, y_test), verbose=2)

Train on 15822 samples, validate on 6782 samples
Epoch 1/100
 - 8s - loss: 0.6985 - acc: 0.5065 - val_loss: 0.6913 - val_acc: 0.5273
Epoch 2/100
 - 7s - loss: 0.6924 - acc: 0.5181 - val_loss: 0.6912 - val_acc: 0.5105
Epoch 3/100
 - 8s - loss: 0.6903 - acc: 0.5242 - val_loss: 0.6880 - val_acc: 0.5516
Epoch 4/100
 - 8s - loss: 0.6891 - acc: 0.5333 - val_loss: 0.6859 - val_acc: 0.5624
Epoch 5/100
 - 8s - loss: 0.6875 - acc: 0.5447 - val_loss: 0.6847 - val_acc: 0.5649
Epoch 6/100
 - 8s - loss: 0.6859 - acc: 0.5521 - val_loss: 0.6844 - val_acc: 0.5532
Epoch 7/100
 - 9s - loss: 0.6848 - acc: 0.5553 - val_loss: 0.6889 - val_acc: 0.5295
Epoch 8/100
 - 8s - loss: 0.6837 - acc: 0.5546 - val_loss: 0.6829 - val_acc: 0.5653
Epoch 9/100
 - 8s - loss: 0.6827 - acc: 0.5553 - val_loss: 0.6830 - val_acc: 0.5631
Epoch 10/100
 - 8s - loss: 0.6819 - acc: 0.5549 - val_loss: 0.6815 - val_acc: 0.5624
Epoch 11/100
 - 8s - loss: 0.6816 - acc: 0.5606 - val_loss: 0.6806 - val_acc: 0.5636
Epoch 12/100
 - 8s - loss

KeyboardInterrupt: 

# looks like extra filters (past 10) don't really help

In [11]:
# Now trying to use an example based on Kim's paper.
# adapted from https://github.com/alexander-rakhlin/CNN-for-Sentence-Classification-in-Keras/blob/master/sentiment_cnn.py

In [18]:
"""
Train convolutional network for sentiment analysis on IMDB corpus. Based on
"Convolutional Neural Networks for Sentence Classification" by Yoon Kim
http://arxiv.org/pdf/1408.5882v2.pdf
For "CNN-rand" and "CNN-non-static" gets to 88-90%, and "CNN-static" - 85% after 2-5 epochs with following settings:
embedding_dim = 50          
filter_sizes = (3, 8)
num_filters = 10
dropout_prob = (0.5, 0.8)
hidden_dims = 50
Differences from original article:
- larger IMDB corpus, longer sentences; sentence length is very important, just like data size
- smaller embedding dimension, 50 instead of 300
- 2 filter sizes instead of original 3
- fewer filters; original work uses 100, experiments show that 3-10 is enough;
- random initialization is no worse than word2vec init on IMDB corpus
- sliding Max Pooling instead of original Global Pooling
"""

import numpy as np

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, GlobalMaxPooling1D, Convolution1D
from keras.layers.merge import Concatenate

np.random.seed(0)

# Model Hyperparameters
embedding_dim = 4
filter_sizes = (3, 4, 5)
num_filters = 10
dropout_prob = 0.5
hidden_dims = 50

# Training parameters
batch_size = 50
num_epochs = 100

# Prepossessing parameters
sequence_length = 181



print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)

# prepare input shape
input_shape = (sequence_length, embedding_dim)
model_input = Input(shape=input_shape)
z = model_input

# Convolutional block
conv_blocks = []
for sz in filter_sizes:
    conv = Convolution1D(filters=num_filters,
                         kernel_size=sz,
                         padding="valid",
                         activation="selu",
                         strides=1)(z)
    conv = GlobalMaxPooling1D()(conv)
    conv_blocks.append(conv)
z = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]

z = Dropout(dropout_prob)(z)
#z = Dense(hidden_dims, activation="selu")(z)
model_output = Dense(1, activation="sigmoid")(z)

model = Model(model_input, model_output)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
model.fit(x_train, y_train, batch_size=batch_size, epochs=num_epochs,
          validation_data=(x_test, y_test), verbose=2)

x_train shape: (15822, 181, 4)
x_test shape: (6782, 181, 4)
Train on 15822 samples, validate on 6782 samples
Epoch 1/100
 - 5s - loss: 0.7457 - acc: 0.5034 - val_loss: 0.6917 - val_acc: 0.5227
Epoch 2/100
 - 3s - loss: 0.6954 - acc: 0.5143 - val_loss: 0.6890 - val_acc: 0.5360
Epoch 3/100
 - 3s - loss: 0.6920 - acc: 0.5245 - val_loss: 0.6884 - val_acc: 0.5382
Epoch 4/100
 - 3s - loss: 0.6907 - acc: 0.5303 - val_loss: 0.6872 - val_acc: 0.5504
Epoch 5/100
 - 3s - loss: 0.6899 - acc: 0.5362 - val_loss: 0.6873 - val_acc: 0.5481
Epoch 6/100
 - 3s - loss: 0.6890 - acc: 0.5363 - val_loss: 0.6864 - val_acc: 0.5516
Epoch 7/100
 - 3s - loss: 0.6882 - acc: 0.5449 - val_loss: 0.6862 - val_acc: 0.5512
Epoch 8/100
 - 4s - loss: 0.6874 - acc: 0.5412 - val_loss: 0.6867 - val_acc: 0.5485
Epoch 9/100
 - 4s - loss: 0.6871 - acc: 0.5454 - val_loss: 0.6850 - val_acc: 0.5509
Epoch 10/100
 - 4s - loss: 0.6864 - acc: 0.5506 - val_loss: 0.6853 - val_acc: 0.5463
Epoch 11/100
 - 4s - loss: 0.6857 - acc: 0.5497 - 

KeyboardInterrupt: 

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(181, 4)))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=50, epochs=10,
          validation_data=(x_test, y_test), verbose=2)

Train on 15822 samples, validate on 6782 samples
Epoch 1/10
 - 10s - loss: 0.6929 - acc: 0.5056 - val_loss: 0.6908 - val_acc: 0.5252
Epoch 2/10
 - 9s - loss: 0.6924 - acc: 0.5157 - val_loss: 0.6900 - val_acc: 0.5276
Epoch 3/10
 - 9s - loss: 0.6916 - acc: 0.5257 - val_loss: 0.6903 - val_acc: 0.5279
Epoch 4/10
 - 9s - loss: 0.6900 - acc: 0.5331 - val_loss: 0.6859 - val_acc: 0.5535
Epoch 5/10
 - 9s - loss: 0.6880 - acc: 0.5390 - val_loss: 0.6860 - val_acc: 0.5522
Epoch 6/10
 - 9s - loss: 0.6863 - acc: 0.5454 - val_loss: 0.6847 - val_acc: 0.5597
Epoch 7/10
 - 9s - loss: 0.6836 - acc: 0.5566 - val_loss: 0.6828 - val_acc: 0.5649
Epoch 8/10
 - 9s - loss: 0.6811 - acc: 0.5607 - val_loss: 0.6805 - val_acc: 0.5681
Epoch 9/10
 - 10s - loss: 0.6786 - acc: 0.5691 - val_loss: 0.6864 - val_acc: 0.5543
Epoch 10/10
 - 9s - loss: 0.6763 - acc: 0.5759 - val_loss: 0.6807 - val_acc: 0.5705


In [20]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

model = Sequential()
model.add(Conv1D(64, 3, activation='selu', input_shape=(181, 4)))
model.add(Conv1D(64, 3, activation='selu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='selu'))
model.add(Conv1D(128, 3, activation='selu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=50, epochs=10,
          validation_data=(x_test, y_test), verbose=2)

Train on 15822 samples, validate on 6782 samples
Epoch 1/10
 - 16s - loss: 0.6971 - acc: 0.5214 - val_loss: 0.6944 - val_acc: 0.5029
Epoch 2/10
 - 15s - loss: 0.6946 - acc: 0.5168 - val_loss: 0.6933 - val_acc: 0.5099
Epoch 3/10
 - 15s - loss: 0.6922 - acc: 0.5257 - val_loss: 0.6902 - val_acc: 0.5218
Epoch 4/10
 - 15s - loss: 0.6923 - acc: 0.5216 - val_loss: 0.6892 - val_acc: 0.5434
Epoch 5/10
 - 15s - loss: 0.6909 - acc: 0.5296 - val_loss: 0.6905 - val_acc: 0.5223
Epoch 6/10
 - 14s - loss: 0.6896 - acc: 0.5373 - val_loss: 0.6919 - val_acc: 0.5159
Epoch 7/10
 - 14s - loss: 0.6899 - acc: 0.5321 - val_loss: 0.6881 - val_acc: 0.5414
Epoch 8/10
 - 14s - loss: 0.6905 - acc: 0.5318 - val_loss: 0.6894 - val_acc: 0.5320
Epoch 9/10
 - 14s - loss: 0.6885 - acc: 0.5422 - val_loss: 0.6868 - val_acc: 0.5569
Epoch 10/10
 - 14s - loss: 0.6892 - acc: 0.5378 - val_loss: 0.6855 - val_acc: 0.5618


In [21]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D

model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(181, 4)))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=50, epochs=10,
          validation_data=(x_test, y_test), verbose=2)

Train on 15822 samples, validate on 6782 samples
Epoch 1/10
 - 12s - loss: 0.6939 - acc: 0.5041 - val_loss: 0.6931 - val_acc: 0.5068
Epoch 2/10
 - 10s - loss: 0.6932 - acc: 0.5027 - val_loss: 0.6930 - val_acc: 0.4937
Epoch 3/10
 - 10s - loss: 0.6920 - acc: 0.5228 - val_loss: 0.6905 - val_acc: 0.5354
Epoch 4/10
 - 10s - loss: 0.6891 - acc: 0.5391 - val_loss: 0.6881 - val_acc: 0.5479
Epoch 5/10
 - 11s - loss: 0.6875 - acc: 0.5463 - val_loss: 0.6884 - val_acc: 0.5472
Epoch 6/10
 - 10s - loss: 0.6853 - acc: 0.5518 - val_loss: 0.6867 - val_acc: 0.5495
Epoch 7/10
 - 10s - loss: 0.6833 - acc: 0.5599 - val_loss: 0.6868 - val_acc: 0.5506
Epoch 8/10
 - 11s - loss: 0.6816 - acc: 0.5590 - val_loss: 0.6874 - val_acc: 0.5444
Epoch 9/10
 - 10s - loss: 0.6791 - acc: 0.5698 - val_loss: 0.6879 - val_acc: 0.5510
Epoch 10/10
 - 10s - loss: 0.6794 - acc: 0.5698 - val_loss: 0.6887 - val_acc: 0.5535
